<a href="https://colab.research.google.com/github/romellfudi/bills_tf_android/blob/master/bentoml_test_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%load_ext watermark
%watermark -a "Romell D.Z." -u -d -p tensorflow,numpy,PIL,keras

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Romell D.Z. 
last updated: 2020-10-02 

tensorflow 2.3.0
numpy 1.18.5
PIL 7.0.0
keras 2.4.3


In [13]:
%%capture
!pip install -q bentoml 
import warnings
warnings.filterwarnings("ignore")

In [2]:
%%capture
!rm -rf bills_photos/ __MACOSX/
!curl -vLJO -H 'Accept: application/octet-stream' https://api.github.com/repos/romellfudi/bills_tf_android/releases/assets/26118074  -u "contactboosttag:254aa92f4c88b57bdbc42070fbd0c66e58d00121" 
!curl -vLJO -H 'Accept: application/octet-stream' https://api.github.com/repos/romellfudi/bills_tf_android/releases/assets/26360766  -u "contactboosttag:254aa92f4c88b57bdbc42070fbd0c66e58d00121" 
!unzip bills_photos.zip
!find . -name '*.xml' -delete

In [3]:
#@title Slipt

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings

os.chdir('bills_photos')
if os.path.isdir('train/10') is False:
    os.makedirs('train/10')
    os.makedirs('train/20')
    os.makedirs('train/50')
    os.makedirs('train/100')
    os.makedirs('valid/10')
    os.makedirs('valid/20')
    os.makedirs('valid/50')
    os.makedirs('valid/100')
    os.makedirs('test/10')
    os.makedirs('test/20')
    os.makedirs('test/50')
    os.makedirs('test/100')

    for i in random.sample(glob.glob('Billetes 10/*.JPG'), 120):
        shutil.move(i, 'train/10')      
    for i in random.sample(glob.glob('Billetes 20/*.JPG'), 120):
        shutil.move(i, 'train/20')   
    for i in random.sample(glob.glob('Billetes 50/*.JPG'), 120):
        shutil.move(i, 'train/50')   
    for i in random.sample(glob.glob('Billetes 100/*.JPG'), 120):
        shutil.move(i, 'train/100')
    for i in random.sample(glob.glob('Billetes 10/*.JPG'), 60):
        shutil.move(i, 'valid/10')   
    for i in random.sample(glob.glob('Billetes 20/*.JPG'), 60):
        shutil.move(i, 'valid/20')   
    for i in random.sample(glob.glob('Billetes 50/*.JPG'), 60):
        shutil.move(i, 'valid/50')        
    for i in random.sample(glob.glob('Billetes 100/*.JPG'), 60):
        shutil.move(i, 'valid/100')
    for i in random.sample(glob.glob('Billetes 10/*.JPG'), 20):
        shutil.move(i, 'test/10')  
    for i in random.sample(glob.glob('Billetes 20/*.JPG'), 20):
        shutil.move(i, 'test/20')  
    for i in random.sample(glob.glob('Billetes 50/*.JPG'), 20):
        shutil.move(i, 'test/50')      
    for i in random.sample(glob.glob('Billetes 100/*.JPG'), 20):
        shutil.move(i, 'test/100')

os.chdir('../')

train_path = 'bills_photos/train'
valid_path = 'bills_photos/valid'
test_path = 'bills_photos/test'

train_batches = ImageDataGenerator(rotation_range=90) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['10', '20', '50', '100'], batch_size=10)
valid_batches = ImageDataGenerator(rotation_range=90) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['10', '20', '50', '100'], batch_size=10)
test_batches = ImageDataGenerator(rotation_range=90) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['10', '20', '50', '100'], batch_size=10, shuffle=False)

Found 480 images belonging to 4 classes.
Found 240 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [205]:
p = 'bills_photos/train/20/*.JPG'
a=!ls {p}
p = a[2].split()[0]
!du -sh {p}

20K	bills_photos/train/20/P9030006.JPG


In [152]:
from tensorflow.keras.models import Sequential, load_model
model = load_model('image_generation_model.h5')
inputs = [] 
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
img = Image.open(p) #TODO
def preprocess_image(image, target_size):
    if image.mode != "RGB":
        image = image.convert("RGB")
    image = image.resize(target_size)
    return np.expand_dims(img_to_array(image), axis=0)
img = preprocess_image(img, target_size=(224, 224)) 
inputs.append(img)
inputs.append(img)
stack = np.vstack(inputs) 
model.predict(stack).tolist()

<class 'numpy.ndarray'>


[[2.1062706666416433e-15, 1.0, 8.008348845178759e-11, 4.712596974573756e-16],
 [2.1062706666416433e-15, 1.0, 8.008348845178759e-11, 4.712596974573756e-16]]

In [203]:
class Bills(tf.keras.Model):
    def __init__(self):
        super(Bills, self).__init__()
        self.cnn = tf.keras. Sequential([
            tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding = 'same', input_shape=(224,224,3)),
            tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),
            tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'),
            tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=4, activation='softmax')
        ])

    @staticmethod
    def image_bytes2tensor(inputs):
        with tf.device("cpu:0"):  # map_fn has issues on GPU https://github.com/tensorflow/tensorflow/issues/28007
            inputs = tf.map_fn(lambda i: tf.io.decode_png(i, channels=3), inputs, dtype=tf.uint8)
        inputs = tf.cast(inputs, tf.float32)
        inputs = (255.0 - inputs) / 255.0
        inputs = tf.reshape(inputs, [-1, 224, 224,3])
        return inputs

    @tf.function(input_signature=[tf.TensorSpec(shape=(None,), dtype=tf.string)])
    def predict_image(self, inputs):
        inputs = self.image_bytes2tensor(inputs)
        return self(inputs)
    
    def call(self, inputs):
        return self.cnn(inputs)

model_ = Bills()

In [209]:
model_.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model_.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=3,
    verbose=1
)

Epoch 1/3
48/48 [==============================] - 47s 971ms/step - loss: 54.4785 - accuracy: 0.3792 - val_loss: 6.1647 - val_accuracy: 0.5292
Epoch 2/3
48/48 [==============================] - 45s 941ms/step - loss: 4.9134 - accuracy: 0.6187 - val_loss: 3.0930 - val_accuracy: 0.6542
Epoch 3/3
48/48 [==============================] - 45s 941ms/step - loss: 4.1381 - accuracy: 0.6292 - val_loss: 2.1595 - val_accuracy: 0.7833


In [200]:
%%writefile tensorflow_bills.py
from typing import List
import io
import base64
from PIL import Image
import numpy as np
from bentoml import BentoService, api, artifacts, env
from bentoml.frameworks.tensorflow import TensorflowSavedModelArtifact
from bentoml.adapters import TfTensorInput, ImageInput, JsonInput
from tensorflow.keras.preprocessing.image import img_to_array

BILLS_CLASSES = ['B10','B20','B50','B100']

# @env(pip_dependencies=['tensorflow', 'numpy', 'pillow'])
@artifacts([TensorflowSavedModelArtifact('model')])
class KerasBillsService(BentoService):
    @api(input=TfTensorInput(), batch=True)
    def predict(self, tensors):
        inputs = []
        for tensor in tensors.numpy():
            image = Image.open(io.BytesIO(tensor))
            if image.mode != "RGB":
                image = image.convert("RGB")
            image = image.resize((224,224 ))
            image = np.expand_dims(img_to_array(image), axis=0)
            inputs.append(image)
        inputs_data = np.vstack(inputs)
        prediction = self.artifacts.model(inputs_data)
        return [(BILLS_CLASSES[np.argmax(i)],i.numpy()) for i in prediction]

Overwriting tensorflow_bills.py


In [210]:
from tensorflow_bills import KerasBillsService
image_generation = KerasBillsService()
image_generation.pack('model', model) # From my repository
# image_generation.pack('model', model_) # it was built here
saved_path = image_generation.save()

[2020-10-02 01:24:24,018] WARNING - pip package requirement tensorflow already exist
INFO:tensorflow:Assets written to: /tmp/bentoml-temp-ux4qri5t/KerasBillsService/artifacts/model_saved_model/assets
[2020-10-02 01:24:25,437] INFO - BentoService bundle 'KerasBillsService:20201002012424_CD4A75' saved to: /root/bentoml/repository/KerasBillsService/20201002012424_CD4A75


In [211]:
!bentoml run KerasBillsService:latest predict --input='{"instances": [{"b64": "iVBORw0KGgoAAAANSUhEUgAAAOAAAADgCAYAAAAaLWrhAAAAAXNSR0IArs4c6QAAAIRlWElmTU0AKgAAAAgABQESAAMAAAABAAEAAAEaAAUAAAABAAAASgEbAAUAAAABAAAAUgEoAAMAAAABAAIAAIdpAAQAAAABAAAAWgAAAAAAAABIAAAAAQAAAEgAAAABAAOgAQADAAAAAQABAACgAgAEAAAAAQAAAOCgAwAEAAAAAQAAAOAAAAAAUWNJNwAAAAlwSFlzAAALEwAACxMBAJqcGAAAAVlpVFh0WE1MOmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDUuNC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8iPgogICAgICAgICA8dGlmZjpPcmllbnRhdGlvbj4xPC90aWZmOk9yaWVudGF0aW9uPgogICAgICA8L3JkZjpEZXNjcmlwdGlvbj4KICAgPC9yZGY6UkRGPgo8L3g6eG1wbWV0YT4KTMInWQAAA4NJREFUeAHt0AENAAAAwqD3T20PBxEoDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDPwMDBENAAFep7MsAAAAAElFTkSuQmCC"},{"b64": "iVBORw0KGgoAAAANSUhEUgAAAOAAAADgCAYAAAAaLWrhAAAAAXNSR0IArs4c6QAAAIRlWElmTU0AKgAAAAgABQESAAMAAAABAAEAAAEaAAUAAAABAAAASgEbAAUAAAABAAAAUgEoAAMAAAABAAIAAIdpAAQAAAABAAAAWgAAAAAAAABIAAAAAQAAAEgAAAABAAOgAQADAAAAAQABAACgAgAEAAAAAQAAAOCgAwAEAAAAAQAAAOAAAAAAUWNJNwAAAAlwSFlzAAALEwAACxMBAJqcGAAAAVlpVFh0WE1MOmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDUuNC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8iPgogICAgICAgICA8dGlmZjpPcmllbnRhdGlvbj4xPC90aWZmOk9yaWVudGF0aW9uPgogICAgICA8L3JkZjpEZXNjcmlwdGlvbj4KICAgPC9yZGY6UkRGPgo8L3g6eG1wbWV0YT4KTMInWQAAA4NJREFUeAHt0AENAAAAwqD3T20PBxEoDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDBgwYMCAAQMGDPwMDBENAAFep7MsAAAAAElFTkSuQmCC"} ]}'

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2020-10-02 01:24:30,744] INFO - Getting latest version KerasBillsService:20201002012424_CD4A75
2020-10-02 01:24:31.312092: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-02 01:24:32.938223: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-02 01:24:32.947893: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-02 01:24:32.947964: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

In [212]:
import base64,json
with open(p, "rb") as f:
    img_bytes = f.read()
img_b64 = base64.b64encode(img_bytes).decode()
data = json.dumps({"instances": [{"b64": img_b64}]})

In [213]:
!bentoml run KerasBillsService:latest predict --input='{data}' 

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2020-10-02 01:24:58,826] INFO - Getting latest version KerasBillsService:20201002012424_CD4A75
2020-10-02 01:24:59.386445: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-02 01:25:00.952543: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-02 01:25:00.962378: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-02 01:25:00.962445: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

In [215]:
p = 'bills_photos/train/100/*.JPG'
p = a[4].split()[0] 
with open(p, "rb") as f:
    img_bytes = f.read()
img_b64 = base64.b64encode(img_bytes).decode()
data = json.dumps({"instances": [{"b64": img_b64}]})
!bentoml run KerasBillsService:latest predict --input='{data}' 

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2020-10-02 01:25:57,856] INFO - Getting latest version KerasBillsService:20201002012424_CD4A75
2020-10-02 01:25:58.445494: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-02 01:26:00.018549: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-02 01:26:00.028452: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-02 01:26:00.028514: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running 

In [ ]:
!bentoml serve KerasBillsService:latest --run-with-ngrok

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2020-10-02 01:26:41,281] INFO - Getting latest version KerasBillsService:20201002012424_CD4A75
[2020-10-02 01:26:41,282] INFO - Starting BentoML API server in development mode..
2020-10-02 01:26:42.012405: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-02 01:26:43.697318: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-02 01:26:43.707379: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-10-02 01:26:43.707438: I tensorflow/stream_

In [ ]:
curl -i \
    --request POST \
    --header "Content-Type: application/json" \
    --data {data} \
    http://b58c50af9d79.ngrok.io/predict

In [ ]:
curl -i \
    --request POST \
    --header "Content-Type: application/json" \
    --data $a \
    http://b58c50af9d79.ngrok.io/predict